In [85]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


In [86]:
import gensim.downloader as api

model = api.load('word2vec-google-news-300')

In [87]:
!pip install contractions
!pip install emoji

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [88]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
%matplotlib inline

import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import contractions
import emoji

from  sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [89]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'d', 'herself', 'by', "shouldn't", 'will', 'such', 'm', "you've", 'why', 'she', "they'd", "she'd", 'does', 'doesn', 'and', 'but', "weren't", 'above', 'down', 'mustn', 'isn', 'of', 'all', 'is', 'couldn', 'itself', 'only', 'o', 'at', "hadn't", 'again', 'below', 'on', "wouldn't", 'those', 'same', 'against', 'aren', 'own', "should've", "she's", 'then', 'do', 'both', 'needn', "we'll", 'because', 'few', 'didn', 's', 'where', 'how', 'we', 'it', 'ourselves', 'be', 'any', 'before', 'don', 'weren', 'each', 'himself', 'was', "he'll", 'wasn', 'once', 'y', 'mightn', 'up', 'from', "we'd", 'with', 'when', 'shouldn', "you'll", 'he', 'hers', 'these', 'too', 'ours', 'other', "aren't", 'what', 't', 'am', "that'll", "you'd", 'should', 'him', 'shan', 'than', 'most', 'the', 'just', "it'll", "hasn't", 'll', 'yours', "you're", 'an', 'this', "we've", 'you', 'yourself', 'can', 'its', 'have', 'or', "haven't", 'their', 'now', 'been', 'ain', 'whom', 'if', "mustn't", 'her', 'theirs', 'until', 'for', 'has', 've', "

In [90]:
df= pd.read_csv("C:/Users/hp/Downloads/Tweets.csv", encoding = "latin-1")
df.head()
df.drop(['tweet_id',"airline_sentiment_confidence", "negativereason", "negativereason_confidence","airline_sentiment_gold", "negativereason_gold", "name", "retweet_count", "tweet_coord", "tweet_created", "tweet_location", "user_timezone"], axis=1, inplace=True)
df.head(100)
df.rename(columns = {'airline_sentiment':'sentiment'},inplace=True)
df.head()

,sentiment,airline,text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...


In [91]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded = encoder.fit_transform(df["sentiment"])
df["encoded"] = encoded

In [92]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
         # Default to noun
        return wordnet.NOUN 

In [93]:
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(text):
    text = text.lower()

    # Expand contractions
    text = contractions.fix(text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove emojis and special characters (optional)
    text = emoji.replace_emoji(text, replace='')

    # Tokenize
    tokens = word_tokenize(text)

    # POS tagging
    pos_tags = pos_tag(tokens)

    # Lemmatize
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]

    # Join back to string
    return ' '.join(lemmatized)

In [94]:
# Apply preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_tweet)
df.head()

,sentiment,airline,text,encoded,cleaned_text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.,1,what say
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...,2,plus you have add commercial to the experience...
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...,1,i do not today must mean i need to take anothe...
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...,0,it be really aggressive to blast obnoxious ent...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...,0,and it be a really big bad thing about it


In [95]:
def remove_stop_words(token):
  return [word.lower() for word in token if word.lower() not in stop_words]

# Only tokenize if not already tokenized
df.iloc[:, 4] = df.iloc[:, 4].apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)
print("dataframe without removing stopping words :\n", df.head())

df.iloc[:, 4] = df.iloc[:, 4].apply(lambda x: remove_stop_words(x) if isinstance(x, list) else x)
print("df after stop_words are removed: \n", df.head())
df.head()
# print(df.iloc[3, 4])

dataframe without removing stopping words :
   sentiment         airline  \
0   neutral  Virgin America   
1  positive  Virgin America   
2   neutral  Virgin America   
3  negative  Virgin America   
4  negative  Virgin America   

                                                text  encoded  \
0                @VirginAmerica What @dhepburn said.        1   
1  @VirginAmerica plus you've added commercials t...        2   
2  @VirginAmerica I didn't today... Must mean I n...        1   
3  @VirginAmerica it's really aggressive to blast...        0   
4  @VirginAmerica and it's a really big bad thing...        0   

                                        cleaned_text  
0                                        [what, say]  
1  [plus, you, have, add, commercial, to, the, ex...  
2  [i, do, not, today, must, mean, i, need, to, t...  
3  [it, be, really, aggressive, to, blast, obnoxi...  
4  [and, it, be, a, really, big, bad, thing, abou...  
df after stop_words are removed: 
   sentiment 

,sentiment,airline,text,encoded,cleaned_text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.,1,[say]
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...,2,"[plus, add, commercial, experience, tacky]"
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...,1,"[today, must, mean, need, take, another, trip]"
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...,0,"[really, aggressive, blast, obnoxious, enterta..."
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...,0,"[really, big, bad, thing]"


In [96]:
def vector_text(model, tokens):
  vectors = [model[word].reshape(1,-1) for word in tokens if word in model]
  if vectors:
    return np.mean(vectors, axis = 0)
  else:
    return np.zeros(300)

df['vector'] = df.iloc[:, 4].apply(lambda x: vector_text(model, x) if isinstance(x, list) else np.zeros(300))
df.head()
# print("Shape of vector for row 0:", df['vector'][0].shape)
# print(df['vector'][0])

,sentiment,airline,text,encoded,cleaned_text,vector
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.,1,[say],"[[-0.036132812, -0.12109375, 0.13378906, 0.114..."
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...,2,"[plus, add, commercial, experience, tacky]","[[0.006713867, -0.027880859, -0.050231934, 0.0..."
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...,1,"[today, must, mean, need, take, another, trip]","[[-0.0025896344, 0.04867118, 0.0355399, 0.0349..."
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...,0,"[really, aggressive, blast, obnoxious, enterta...","[[0.0012191773, 0.09075928, -0.017980957, 0.05..."
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...,0,"[really, big, bad, thing]","[[0.11010742, 0.06271362, 0.0031738281, 0.1318..."


In [97]:
X = np.vstack(df['vector'].values)
y = df['encoded'].to_numpy()
print(y.shape)
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(X_train[0].shape)
# print(y_train[0].shape)
# print(X_train.shape)
# print(y_train.shape)

(14640,)
(14640, 300)


In [98]:
from sklearn.linear_model import LogisticRegression
modell = LogisticRegression(multi_class='multinomial',random_state = 42, max_iter = 1000)
modell.fit(X_train, y_train)

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [99]:
y_pred = modell.predict(X_test)
print(y_pred[-30:])
print(y_test[-30:])

[0 2 0 0 2 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 2 0 2 2 0 2 0 0 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 1 1 0]


In [100]:
print(f"Accuracy:  {accuracy_score(y_test, y_pred)*100 :.2f}%")
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))

Accuracy:  77.49%
              precision    recall  f1-score   support

    negative       0.81      0.92      0.86      1889
     neutral       0.59      0.43      0.50       580
    positive       0.76      0.62      0.69       459

    accuracy                           0.77      2928
   macro avg       0.72      0.66      0.68      2928
weighted avg       0.76      0.77      0.76      2928



In [101]:
# import tensorflow as tf
# from tensorflow import keras

# num_outputs = 3
# tf.random.set_seed(1)

# NN = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(512, activation = "relu"),
#     tf.keras.layers.Dense(256, activation = "relu"),
#     tf.keras.layers.Dense(128, activation = "relu"),
#     tf.keras.layers.Dense(64, activation = "relu"),
#     tf.keras.layers.Dense(num_outputs, activation = "softmax")
# ])

# NN.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy', 
#     metrics=['accuracy']
# )

# history = NN.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1)
# test_loss, test_acc = NN.evaluate(X_test, y_test)
# print(f"Test Accuracy: {test_acc*100 :.2f}%" )

# y_pred = NN.predict(X_test)
# predicted_classes = np.argmax(y_pred, axis=1)
# true_classes = np.argmax(y_test, axis=1)

In [102]:
# print(y_pred)
# label_map = np.array(["negative", "neutral", "positive"])
# sentiments = label_map[true_classes]
# print(sentiments)

In [103]:
# print(y_test)

In [105]:
def text_vector(w2v_model, message):
    tokens = word_tokenize(message)
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    token_vectors = [w2v_model[word] for word in tokens if word in w2v_model]
    if token_vectors:
        return np.mean(token_vectors, axis = 0)
    else:
        return np.zeros(300)

def predict_tweet_sentiment(classifier, w2v_model, message):
    
    vector = text_vector(w2v_model, message).reshape(1, -1)
    prediction = classifier.predict(vector)

    if prediction == 0:
        return "negative"
    elif prediction ==1:
        return "neutral"
    else:
        return "positive"

message = "I lost my luggage and customer support didn’t respond."
print(predict_message_class(modell, model, message))

message = "The flight attendants were incredibly kind and helpful."
print(predict_message_class(modell, model, message))
message = "The flight was on time and everything went as expected."
print(predict_message_class(modell, model, message))

message = "The cabin crew did their job but weren't overly friendly."
print(predict_message_class(modell, model, message))

message = "The food was okay, not too bad, not great either."
print(predict_message_class(modell, model, message))

message = "I had a smooth flight and landed earlier than expected."
print(predict_message_class(modell, model, message))

message = "The boarding process was fast and well organized."
print(predict_message_class(modell, model, message))

negative
negative
negative
positive
negative
positive
negative
